importa os arquivos csv historicos dos titulos e exporta no bD

In [67]:
import pandas as pd

# Definir o titulo ao importar
id_titulos = 1  # Substitua pelo valor desejado

# Ler o arquivo CSV do titulo
df = pd.read_csv("dollar-index-forecast-data.csv")
# remover a ultima linea
df = df.iloc[:-1]
#Remover espacios em branco
df.iloc[:, 0] = df.iloc[:, 0].str.strip()


# Converter a primeira coluna para datetime (supondo que seja no formato 'mmm/yyyy')
df.iloc[:, 0] = pd.to_datetime(df.iloc[:, 0],format="%b %Y", errors="coerce")

# Renomear a primeira coluna para 'data'
df.rename(columns={df.columns[0]: "data"}, inplace=True)

# Converter a segunda coluna para float e renomeá-la para 'valor'
df.iloc[:, 1] = df.iloc[:, 1].astype(float)
df.rename(columns={df.columns[1]: "valor"}, inplace=True)

# Criar a terceira coluna com o valor de id_titulos
df["id_titulos"] = id_titulos

print(df)

                   data    valor       Note  id_titulos
0   2020-04-01 00:00:00   99.900     Actual           1
1   2020-05-01 00:00:00   99.570     Actual           1
2   2020-06-01 00:00:00   97.102     Actual           1
3   2020-07-01 00:00:00   95.660     Actual           1
4   2020-08-01 00:00:00   93.056     Actual           1
..                  ...      ...        ...         ...
64  2025-08-01 00:00:00  104.200   Forecast           1
65  2025-09-01 00:00:00  107.420   Forecast           1
66  2025-10-01 00:00:00  108.190   Forecast           1
67  2025-11-01 00:00:00  106.770   Forecast           1
68  2025-12-01 00:00:00  103.830   Forecast           1

[69 rows x 4 columns]


In [69]:
df = df.drop(df[df.iloc[:, 2] == " Forecast"].index)
print(df)

                   data    valor     Note  id_titulos
0   2020-04-01 00:00:00   99.900   Actual           1
1   2020-05-01 00:00:00   99.570   Actual           1
2   2020-06-01 00:00:00   97.102   Actual           1
3   2020-07-01 00:00:00   95.660   Actual           1
4   2020-08-01 00:00:00   93.056   Actual           1
..                  ...      ...      ...         ...
56  2024-12-01 00:00:00  107.173   Actual           1
57  2025-01-01 00:00:00  108.586   Actual           1
58  2025-02-01 00:00:00  107.389   Actual           1
59  2025-03-01 00:00:00  104.207   Actual           1
60  2025-04-01 00:00:00  100.725   Actual           1

[61 rows x 4 columns]


In [74]:
import sqlite3

# Conectar ao banco de dados SQLite
conn = sqlite3.connect("sqforecasts.db")
cursor = conn.cursor()

# Converter a coluna "data" do DataFrame para string
df["data"] = df["data"].astype(str)

# Inserir os dados do DataFrame na tabela tbtituloshist
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO tbtituloshist (id_titulos, data, valor) 
        VALUES (?, ?, ?)
    """, (row["id_titulos"], row["data"], row["valor"]))

# Salvar e fechar conexão
conn.commit()
conn.close()

print("Dados exportados com sucesso para a tabela tbtituloshist!")

Dados exportados com sucesso para a tabela tbtituloshist!
